In [1]:
# %load_ext autoreload
# %autoreload 2

# import importlib

import numpy as np

import torch
import torchvision.models.resnet as resnet
from torch.utils.data import DataLoader, TensorDataset
from torch.backends import cudnn
cudnn.benchmark = True

from stylegan2.solver import Solver

In [2]:
class ResNet(resnet.ResNet):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.sigmoid(self._forward_impl(x))

In [3]:
batch_size = 16
num_workers = 1

X_val = torch.from_numpy((np.concatenate([np.load('X0_val_clean_48.npy')[:, :, 8:40, 8:40], 
                                          np.load('X1_val_clean_48.npy')[:, :, 8:40, 8:40]]) - .5) / .5)
y_val = torch.from_numpy(np.concatenate([np.load('y0_val_clean_48.npy'), np.load('y1_val_clean_48.npy')]))

valloader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True, drop_last=True)

Clf = ResNet(resnet.BasicBlock, [2, 2, 2, 2], num_classes=1)
Clf.load_state_dict(torch.load('results/clf_resnet18_32/best_model.pth'));

X_train = (np.concatenate([np.load('X0_train_clean_48.npy'), np.load('X1_train_clean_48.npy')]) - .5) / .5

trainloader = DataLoader(torch.from_numpy(X_train[:, :, 8:40, 8:40]), batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True)
X_train = None

In [4]:
solver = Solver('StyleGAN_cam48', '0', 32)

In [ ]:
solver.train(trainloader, batch_size, Clf, valloader)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: array (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.19 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


In [7]:
import math
int(math.log(32, 2))

5

In [11]:
list(range(5, 2, -1))

[5, 4, 3]

In [10]:
2**4, 2**3, 2**2

(16, 8, 4)